# Trying Another (Possible?) Code for Identifying Clusters

First, we import Acqu (which parses the events?) and Timepix (which decodes the data??), as well as numpy, ROOT, and plotting classes of ROOT which allow us to organize and graph the data.

In [1]:
import Acqu as aq
import Timepix
import numpy as np
import ROOT
from rootpy.plotting import Hist, Hist2D, histogram, Canvas
from ipykernel import kernelapp as app

Welcome to JupyROOT 6.16/00


Next, we import the data that was collected and open that data file.


In [2]:
inFile = '/w/work0/mainz/2019_05_Timepix3-Acqu/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


In [4]:
TimepixAData = []
TimepixBData = []

def plotTimepix():
    if(aq.epicsEvent==1):
        nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
        encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
        nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')
        encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
        TimepixAData.append(Timepix.Decode(nHitsA,encodedA))
        TimepixBData.append(Timepix.Decode(nHitsB,encodedB))       
aq.runFunction(plotTimepix,0,50000)

print(TimepixAData)

[array([( 14,  25,    12700, 15,  49), (171, 114,    37385, 20,  45),
       (171, 115,    37386, 26,  45), ..., (192, 149, 12867010, 14, 139),
       (229,  35, 12891359, 24,   4), (230,  35, 12891356, 20,  93)],
      dtype=[('x', 'u1'), ('y', 'u1'), ('ToA', '<u4'), ('FToA', 'u1'), ('ToT', '<u2')]), array([( 14,  25,    12700, 15,  49), (171, 114,    37385, 20,  45),
       (171, 115,    37386, 26,  45), ..., (192, 149, 12867010, 14, 139),
       (229,  35, 12891359, 24,   4), (230,  35, 12891356, 20,  93)],
      dtype=[('x', 'u1'), ('y', 'u1'), ('ToA', '<u4'), ('FToA', 'u1'), ('ToT', '<u2')]), array([(129,  98,     5485, 12, 102), (132, 122,     7218, 28,   8),
       (135, 124,     7217, 15,  15), ..., (192, 246, 12826325, 13, 107),
       (215,  60, 12837604,  7,  17), (214,  60, 12837604,  8,  42)],
      dtype=[('x', 'u1'), ('y', 'u1'), ('ToA', '<u4'), ('FToA', 'u1'), ('ToT', '<u2')]), array([(184,  30,    13832, 15,  51), (184,  29,    13832, 17, 102),
       (254, 189,    273

In [7]:
print(TimepixAData[0]['ToA'])
print(TimepixAData[2]['ToA'])

[   12700    37385    37386 ... 12867010 12891359 12891356]
[    5485     7218     7217 ... 12826325 12837604 12837604]


Depending on how the data is structured, maybe we want to create lists of specific things:

In [9]:
ToAAData = []
ToAA = []

for i in range(len(TimepixAData)):
    ToAAData.append(TimepixAData[i]['ToA'])
for j in range(len(ToAAData)):
    for k in range(len(ToAAData[j])):
        ToAA.append(ToAAData[j][k])

ToAA.sort()
        
ToABData = []
ToAB = []

for i in range(len(TimepixBData)):
    ToABData.append(TimepixBData[i]['ToA'])
for j in range(len(ToABData)):
    for k in range(len(ToABData[j])):
        ToAB.append(ToABData[j][k])

ToAB.sort()

print(ToAA)

[147, 183, 184, 184, 185, 186, 304, 304, 308, 308, 387, 405, 407, 661, 661, 1576, 1576, 1577, 1577, 1577, 1577, 1577, 1577, 1577, 1577, 1577, 1577, 1578, 1578, 1579, 1579, 1739, 3090, 3090, 3435, 3435, 3436, 3436, 3576, 3576, 3716, 3716, 3717, 3717, 3718, 3718, 3725, 3725, 3727, 3864, 3867, 4009, 4009, 4010, 4010, 4084, 4085, 4165, 4165, 4166, 4166, 4589, 4589, 4810, 4810, 4811, 4811, 4811, 4811, 4811, 4811, 4811, 4811, 4824, 4826, 5034, 5347, 5347, 5349, 5349, 5485, 5494, 5494, 5494, 5494, 5494, 5494, 5494, 5494, 5494, 5494, 5494, 5494, 5733, 5733, 5735, 5957, 6208, 6210, 7216, 7216, 7217, 7217, 7217, 7217, 7218, 7619, 7672, 7672, 7922, 7922, 7923, 7923, 7924, 7924, 8494, 8494, 8494, 8494, 9059, 9059, 9059, 9059, 9643, 9698, 9698, 9699, 9700, 10827, 10827, 10828, 10828, 11831, 11831, 12700, 12700, 13201, 13203, 13832, 13832, 13832, 13832, 14695, 14695, 14698, 14698, 16081, 16726, 16726, 17708, 18272, 18272, 18274, 18274, 18495, 18496, 18985, 21345, 21346, 21513, 21513, 21514, 21514, 2

At some point we'll need to process/analyze the cluster we have (note that this currently only deals with the A detector's data):

In [ ]:
def process_cluster():
    int start_index = cl[0]                # start index of the first hit in the cluster
    float min_time = ToAA[start_index]     # time of the first hit in the cluster
    

Now let's try to identify the clusters. Note that this currently only deals with the A detector's data.

In [19]:
cl = [None]*100          # a cluster can hold up to 100 hits
cn = 0                   # counts number of hits in current cluster
blen = 0                 # the length of the current tp3 buffer (?)
b = 0                    # the running (current?) buffer position

blen = 100 #readTimepixBufferFromFile() - but this has no definition
while(blen): # read each timpix buffer into tp3?
    
    while(b<blen):        # for every buffer position in the buffer?
        cn = 0            # zero the cluster counter for the current buffer position
        cl[cn] = b        # assume the current position has the first hit in the cluster and save it#
        cn+=1             # increment the counter, since we've counted a hit
        
        # this next bit was a for loop in Ken's code
        c=b+1             # start at position c that is one above the current buffer position
        while((c<b+99) and (c<blen)):           # check all positions up to 99 away, as long as we're still in the buffer
            if(abs(ToAA[c] - ToAA[b] < 100)):   # if an event at c is within 100ns of that at the current buffer position
               cl[cn] = c                       # save this position to the cluster
               cn+=1                            # increment the counter, since we've counted a hit
            else:
               #process_cluster()               # analyze the cluster we found
               b+=cn                            # move on beyond this cluster
            
            c+=1 # move to the next position and go through this loop again

36
37
37
38
39
157
none here!
157
none here!
161
none here!
161
none here!
240
none here!
258
none here!
260
none here!
514
none here!
514
none here!
1429
none here!
1429
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1431
none here!
1431
none here!
1432
none here!
1432
none here!
1592
none here!
2943
none here!
2943
none here!
3288
none here!
3288
none here!
3289
none here!
3289
none here!
3429
none here!
3429
none here!
3569
none here!
3569
none here!
3570
none here!
3570
none here!
3571
none here!
3571
none here!
3578
none here!
3578
none here!
3580
none here!
3717
none here!
3720
none here!
3862
none here!
3862
none here!
3863
none here!
3863
none here!
3937
none here!
3938
none here!
4018
none here!
4018
none here!
4019
none here!
4019
none here!
4442
none here!
4442
none here!
4663
none here!
4663
none here!
4664
none here!
4664
none here!
4664
none here!
46

none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1431
none here!
1431
none here!
1432
none here!
1432
none here!
1592
none here!
2943
none here!
2943
none here!
3288
none here!
3288
none here!
3289
none here!
3289
none here!
3429
none here!
3429
none here!
3569
none here!
3569
none here!
3570
none here!
3570
none here!
3571
none here!
3571
none here!
3578
none here!
3578
none here!
3580
none here!
3717
none here!
3720
none here!
3862
none here!
3862
none here!
3863
none here!
3863
none here!
3937
none here!
3938
none here!
4018
none here!
4018
none here!
4019
none here!
4019
none here!
4442
none here!
4442
none here!
4663
none here!
4663
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4677
none here!
4679
none here!
4887
none here!
5200
none here!
5200
none here!
5202
none here!
5202
none here!
5338
none here!
5347
none her

none here!
5347
none here!
5347
none here!
5586
none here!
5586
none here!
5588
none here!
5810
none here!
6061
none here!
6063
none here!
36
37
37
38
39
157
none here!
157
none here!
161
none here!
161
none here!
240
none here!
258
none here!
260
none here!
514
none here!
514
none here!
1429
none here!
1429
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1431
none here!
1431
none here!
1432
none here!
1432
none here!
1592
none here!
2943
none here!
2943
none here!
3288
none here!
3288
none here!
3289
none here!
3289
none here!
3429
none here!
3429
none here!
3569
none here!
3569
none here!
3570
none here!
3570
none here!
3571
none here!
3571
none here!
3578
none here!
3578
none here!
3580
none here!
3717
none here!
3720
none here!
3862
none here!
3862
none here!
3863
none here!
3863
none here!
3937
none here!
3938
none here!
4018
none here!
4018
none here!
4019
no

none here!
1592
none here!
2943
none here!
2943
none here!
3288
none here!
3288
none here!
3289
none here!
3289
none here!
3429
none here!
3429
none here!
3569
none here!
3569
none here!
3570
none here!
3570
none here!
3571
none here!
3571
none here!
3578
none here!
3578
none here!
3580
none here!
3717
none here!
3720
none here!
3862
none here!
3862
none here!
3863
none here!
3863
none here!
3937
none here!
3938
none here!
4018
none here!
4018
none here!
4019
none here!
4019
none here!
4442
none here!
4442
none here!
4663
none here!
4663
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4677
none here!
4679
none here!
4887
none here!
5200
none here!
5200
none here!
5202
none here!
5202
none here!
5338
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none her

4679
none here!
4887
none here!
5200
none here!
5200
none here!
5202
none here!
5202
none here!
5338
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5586
none here!
5586
none here!
5588
none here!
5810
none here!
6061
none here!
6063
none here!
36
37
37
38
39
157
none here!
157
none here!
161
none here!
161
none here!
240
none here!
258
none here!
260
none here!
514
none here!
514
none here!
1429
none here!
1429
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1431
none here!
1431
none here!
1432
none here!
1432
none here!
1592
none here!
2943
none here!
2943
none here!
3288
none here!
3288
none here!
3289
none here!
3289
none here!
3429
none here!
3429
none here!
3569
none here!
3569
none here!
3570
none here!
35

...

none here!
514
none here!
514
none here!
1429
none here!
1429
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1431
none here!
1431
none here!
1432
none here!
1432
none here!
1592
none here!
2943
none here!
2943
none here!
3288
none here!
3288
none here!
3289
none here!
3289
none here!
3429
none here!
3429
none here!
3569
none here!
3569
none here!
3570
none here!
3570
none here!
3571
none here!
3571
none here!
3578
none here!
3578
none here!
3580
none here!
3717
none here!
3720
none here!
3862
none here!
3862
none here!
3863
none here!
3863
none here!
3937
none here!
3938
none here!
4018
none here!
4018
none here!
4019
none here!
4019
none here!
4442
none here!
4442
none here!
4663
none here!
4663
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4677
none here!
4679
none here!

39
157
none here!
157
none here!
161
none here!
161
none here!
240
none here!
258
none here!
260
none here!
514
none here!
514
none here!
1429
none here!
1429
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1431
none here!
1431
none here!
1432
none here!
1432
none here!
1592
none here!
2943
none here!
2943
none here!
3288
none here!
3288
none here!
3289
none here!
3289
none here!
3429
none here!
3429
none here!
3569
none here!
3569
none here!
3570
none here!
3570
none here!
3571
none here!
3571
none here!
3578
none here!
3578
none here!
3580
none here!
3717
none here!
3720
none here!
3862
none here!
3862
none here!
3863
none here!
3863
none here!
3937
none here!
3938
none here!
4018
none here!
4018
none here!
4019
none here!
4019
none here!
4442
none here!
4442
none here!
4663
none here!
4663
none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here

5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5586
none here!
5586
none here!
5588
none here!
5810
none here!
6061
none here!
6063
none here!
36
37
37
38
39
157
none here!
157
none here!
161
none here!
161
none here!
240
none here!
258
none here!
260
none here!
514
none here!
514
none here!
1429
none here!
1429
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1431
none here!
1431
none here!
1432
none here!
1432
none here!
1592
none here!
2943
none here!
2943
none here!
3288
none here!
3288
none here!
3289
none here!
3289
none here!
3429
none here!
3429
none here!
3569
none here!
3569
none here!
3570
none here!
3570
none here!
3571
none here!
3571
none here!
3578
none here!
3578
none here!
3580
none here!
3717
none here!
3720
none here!
3862
none here!
3862
none here!
3863
none here!
3863
none here!
39

none here!
4664
none here!
4664
none here!
4664
none here!
4664
none here!
4677
none here!
4679
none here!
4887
none here!
5200
none here!
5200
none here!
5202
none here!
5202
none here!
5338
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5347
none here!
5586
none here!
5586
none here!
5588
none here!
5810
none here!
6061
none here!
6063
none here!
36
37
37
38
39
157
none here!
157
none here!
161
none here!
161
none here!
240
none here!
258
none here!
260
none here!
514
none here!
514
none here!
1429
none here!
1429
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1430
none here!
1431
none here!
1431
none here!
1432
none here!
1432
none here!
1592
none here!
2943
none here!
2943
none here!
3288
none here!
3288
none here!
3289
none here!
3289
no

KeyboardInterrupt: 

Note to self: rewrite in the main code step by step, and also process the cluster